# **XGBoost를 이용한 사출공정의 최적 공정조건 산출**

## [단계 1] 라이브러리/데이터 불러오기

### ① 패키지 설치하기

패키지를 설치하기에 앞서 사용자의 작업환경에 설치된 패키지 목록을 확인합니다.

In [ ]:
!pip list

*pip install* 명령어를 이용하여 분석에 필요한 패키지를 설치합니다.

In [ ]:
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install pathlib
!pip install datetime
!pip install parse
!pip install sklearn
!pip install mlxtend --upgrade --no-deps
!pip install xgboost
!pip install pyswarm

### ② 패키지 및 모듈 불러오기

In [ ]:
import numpy as np
import pandas as pd
import argparse
from pathlib import Path
from numpy import mean
from datetime import datetime
from pathlib import Path
from numpy import mean
from datetime import datetime
from parse import compile
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from mlxtend.frequent_patterns import fpgrowth
import xgboost
from xgboost import plot_importance
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pyswarm import pso
import GPUtil
warnings.simplefilter("ignore")

### ③ 데이터 불러오기

In [ ]:
path = './raw_data.csv'
df_raw = pd.read_csv(path)

## [단계 2] 데이터 종류 및 개수 확인

### ① 데이터의 구성 확인

In [ ]:
print(df_raw.head())
print(df_raw.info())

## [단계 3] 데이터 전처리


### ① 렌즈 분류 및 카테고리 열 추가

분석 모델 개발 과정에서 렌즈의 종류를 분리하여 모델을 생성했을 때 예측력이 더 높아졌기 때문에 도수에 따라 카테고리를 만들어 데이터셋에 추가한다.

데이터가 가장 많은 근시(MYOPIA)를 CATEGORY열의 기본값으로 추가한 후 REAL_POWER 값이 0과 같으면 BEAUTY, 0보다 크면 HYPER로 값을 변경한다.

In [ ]:
df_raw['CATEGORY'] = 'MYOPIA'
df_raw['CATEGORY'][df_raw['REAL_POWER'] == 0] = 'BEAUTY'
df_raw['CATEGORY'][df_raw['REAL_POWER'] > 0] = 'HYPER'

print(df_raw.info())

### ② 자주 사용하는 변수 정의

In [ ]:
# 금형조합과 곡률
CORES = ['MOLD_IN_TOP','MOLD_IN_BOT','MOLD_OUT_TOP','MOLD_OUT_BOT']
RADIUS = ['IN_RADIUS','OUT_RADIUS']
EQUIP_INFO = ['EQUIP_ID','MOLD_POS']
POWERS = ['POWER1','POWER2','POWER3','POWER4','POWER5']

### ③ 누락 데이터 처리 함수 정의

NULL이거나 공백인 데이터의 값을 np.NaN으로 수정하고, MOLD_IN_TOP, MOLD_IN_BOT, MOLD_OUT_TOP, MOLD_OUT_BOT 값이 np.NaN인 데이터를 삭제한다.

In [ ]:
def preprocessing_missing(df_before):

    type = 'Missing Value'
    print(f'Type of preprocessing: {type}')
    print(f'The shape of data before preprocessing: {df_before.shape}')

    tmp = np.where((df_before.values == 'NULL')|(df_before.values == ''), np.NaN, df_before.values)
    df_after = pd.DataFrame(data=tmp, columns=df_before.columns)
    df_after = df_after.dropna(how='any', subset=CORES+RADIUS)

    print(f'The shape of data after preprocessing: {df_after.shape}\n')
    
    return df_after

### ④ 데이터 형변환 처리 함수 정의

열(column)의 이름을 모두 대문자로 변환하고 각 데이터의 형을 통일시킨다.

IN_RADIUS와 OUT_RADIUS의 플러스/마이너스 부호 오표기로 인한 에러를 줄이기 위해 절대값으로 수정한다.

In [ ]:
def preprocessing_conversion(df_before):

    type = 'Data Type Conversion'
    print(f'Type of preprocessing: {type}')
    print(f'The shape of data before preprocessing: {df_before.shape}')
    #컬럼명을 모두 대문자로 변환
    df_before.columns = [col.upper() for col in df_before.columns]
    print(df_before.columns)

    # 컬럼의 데이터형 통일
    col_type = {
        'MOLD_POS':'int',
        'REAL_POWER':'float32',
        'CP':'float32',
        'AX':'float32',
        'OUT_RADIUS':'float32',
        'IN_RADIUS':'float32',
        'POWER':'float32',
        'POWER1':'float32',
        'POWER2':'float32',
        'POWER3':'float32',
        'POWER4':'float32',
        'POWER5':'float32'
    }

    # 
    df_after = df_before.astype(col_type)
    df_after['MFG_DT'] = pd.to_datetime(df_before['MFG_DT'])
    
    # IN_RADIUS와 OUT_RADIUS의 값의 부호 삭제
    df_after['IN_RADIUS'] = abs(df_after['IN_RADIUS'])
    df_after['OUT_RADIUS'] = abs(df_after['OUT_RADIUS'])

    # CORES 대문자로 변환
    df_after[CORES] = df_after[CORES].applymap(lambda x: x.upper())


    print(f'The shape of data after preprocessing: {df_after.shape}\n')
    
    return df_after

### ⑤ 지정 날짜 기준 데이터 추출 함수 정의

데이터의 품질이 보장되는 2020년 1월 1일 이후 데이터만 분석에 활용하기 위해 기준일 이후 데이터를 삭제한다.

In [ ]:
def preprocessing_date(df_before):

    type = 'Removing Irrelevant Data in terms of Date'
    print(f'Type of preprocessing: {type}')
    print(f'The shape of data before preprocessing: {df_before.shape}')
    df_after = df_before[df_before['MFG_DT'] >= np.datetime64('2020-01-01')]
    print(f'The shape of data after preprocessing: {df_after.shape}\n')
    
    return df_after

### ⑥ 이상치 처리 함수 정의

POWER는 POWER1, POWER2, POWER3, POWER4, POWER5의 평균을 도수기준표에 넣어 결정하는 값으로 최종 결정된 도수인 REAL_POWER와 일치해야한다.

위 조건을 만족하지 않는 데이터를 제거한 뒤 IN_RADIUS와 OUT_RADIUS를 기준으로 Isolation Forest 알고리즘을 사용하여 이상치를 제거한다.

IN_RADIUS와 OUT_RADIUS를 기준으로 한 산점도를 그려 이상치 제거가 잘 되었는지 확인한다.

In [ ]:
def preprocessing_outlier(df_before):

    type = 'Removing Outliers'
    print(f'Type of preprocessing: {type}')
    print(f'The shape of data before preprocessing: {df_before.shape}')

    # 조건1 : REAL_POWER 값과 POWER 값이 같은 경우
    mask1 = (df_before['REAL_POWER'] == df_before['POWER'])
    # 조건2 : POWER1, POWER2, POWER3, POWER4, POWER5의 평균과 POWER의 오차가 0.5보다 큰 경우
    mask2 = abs(mean(df_before[POWERS], axis=1) - df_before['POWER']) < 0.5
    # 조건1과 조건2를 모두 만족하는 경우만 추출
    df_after = df_before[mask1 & mask2]

    # IsolationForest 적용 전의 데이터 시각화
    fig = plt.figure(figsize=(16,16))
    ax_before = fig.add_subplot(1, 1, 1)
    df_after.plot.scatter(
        x='IN_RADIUS',
        y='OUT_RADIUS',
        ax=ax_before
    )
    
    # IsolationForest 적용 후의 데이터 시각화
    results = IsolationForest(random_state=0).fit_predict(df_after[RADIUS])
    df_after = df_after[results == 1]
    df_after.plot.scatter(
        x='IN_RADIUS',
        y='OUT_RADIUS',
        ax=ax_before,
        color='black'
    )

    plt.show()

    print(f'The shape of data after preprocessing: {df_after.shape}\n')
    
    return df_after

### ⑦ 데이터 전처리 실행

이후 단계는 근시렌즈에 대한 분석과 예측을 진행한다.

앞서 정의한 전처리 프로세스를 순차적으로 실행하고 최종 전처리 데이터를 추출한다.

In [ ]:
df_raw = df_raw[df_raw['CATEGORY']=='MYOPIA']

In [ ]:
df_after = preprocessing_missing(df_raw)

In [ ]:
df_after = preprocessing_conversion(df_after)

In [ ]:
df_after = preprocessing_date(df_after)

In [ ]:
df_after = preprocessing_outlier(df_after)

In [ ]:
print(df_after.info())

## [단계 4] 데이터 특성 파악

### ① describe 함수를 통한 요약통계량 파악

In [ ]:
df_after.describe()

### ② corr 함수를 통한 변수 간 상관관계 파악

In [ ]:
plt.subplots(figsize=(25,25))
sns.heatmap(data = df_after.corr(), linewidths=0.1, annot=True, fmt ='.4f', cmap='Blues')

## [단계 5] XGBoost 모델 학습

### ① 주요 변수 및 함수 정의

In [ ]:
# Onehot encoder
ENCODER = None

# XGBoost
N_ESTIMATORS = 6000
ETA = 0.25
SUB_SAMPLE = 0.9

In [ ]:
def prepare_data(df, numeric_cols, onehot_cols, y_col):
    x_cols = numeric_cols + onehot_cols
    df = df.dropna(
        axis=0,
        how='any',
        subset=x_cols+y_col
    )    
    data_X = df[x_cols]
    data_y = df[y_col]

    print('\nSplitting data into train set and test set...')
    X_train, X_test, y_train, y_test = train_test_split(data_X,data_y,test_size=.3,random_state=42)

    print(f'The shape of X_train, y_train: {X_train.shape}, {y_train.shape}')
    print(f'The shape of X_test, y_test: {X_test.shape}, {y_test.shape}')

    bounds = df.groupby(['REAL_POWER'])[numeric_cols].agg(['min', 'max'])

    return X_train, X_test, y_train, y_test, bounds

In [ ]:
def transform_cols(df, encoder):
    if encoder is None:
        print('ENCODER fit_transform')
        encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
        onehot = encoder.fit_transform(df[EQUIP_INFO])
    else:
        print('ENCODER transform')
        onehot = encoder.transform(df[EQUIP_INFO])

    transformed_onehot_cols = encoder.get_feature_names(EQUIP_INFO).ravel().tolist()
    df = np.hstack((df[RADIUS].to_numpy(),onehot))
    df = pd.DataFrame(df, columns=RADIUS+transformed_onehot_cols)
    
    return df, encoder

### ② 학습 데이터와 평가 데이터 분리

In [ ]:
df_prep = df_after.reset_index().melt(
    id_vars=['index','SALE_CD','REAL_POWER','CP','AX']+RADIUS+EQUIP_INFO,
    value_vars=POWERS,
    value_name='y'
)

print(f'The shape of data which will be returned: {df_prep.shape}')

In [ ]:
df_prep

In [ ]:
print(df_prep.info())

In [ ]:
X_train, X_test, y_train, y_test, bounds = prepare_data(
    df = df_prep,
    numeric_cols = RADIUS,
    onehot_cols = EQUIP_INFO,
    y_col=['y']
)

### ③ XGBoost Regressor 호출 및 파라미터 설정

In [ ]:
if len(GPUtil.getGPUs()) < 1:
    regressor = xgboost.XGBRegressor(
        n_estimators = N_ESTIMATORS,
        objective = 'reg:squarederror',
        eta = ETA,
        tree_method = 'hist',
        gpu_id = 0,
        subsample = SUB_SAMPLE
    )
else:
    regressor = xgboost.XGBRegressor(
        n_estimators = N_ESTIMATORS,
        objective = 'reg:squarederror',
        eta = ETA,
        tree_method = 'gpu_hist',
        gpu_id = 0,
        subsample = SUB_SAMPLE
    )

### ④ X_train과 X_test 열의 One-hot 인코딩

In [ ]:
X_train, ENCODER = transform_cols(X_train, ENCODER)
X_test, ENCODER = transform_cols(X_test, ENCODER)

In [ ]:
X_train.columns

### ⑤ XGBoost Regressor 모델 학습

In [ ]:
history=regressor.fit(
    X = X_train,
    y = y_train,
    eval_set = [(X_train, y_train), (X_test, y_test)], 
    eval_metric = 'mae',
    early_stopping_rounds = N_ESTIMATORS/10
)

In [ ]:
print(f'Score: train({regressor.score(X_train, y_train)}),test({regressor.score(X_test, y_test)})')

### ⑥ 모델 평가

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
plot_importance(regressor, ax=ax)

In [ ]:
plt.figure(figsize=(10,7))
plt.yscale('log')
plt.text(5000, 0.09, "Minimum mae \n= "+str(min(xgb_val["validation_0"]["mae"])))
plt.plot(xgb_val["validation_0"]["mae"], label='Training loss')
plt.plot(xgb_val["validation_1"]["mae"], label='Validation loss')
plt.axvline(regressor.best_ntree_limit, color='gray', label='Optimal tree number')
plt.xlabel('Number of trees')
plt.ylabel('Loss(log)')
plt.legend()

plt.show()

## [단계 6] 목표도수를 생산하기 위핸 최적 금형조합 도출

### ① 결과 데이터프레임 생성

In [ ]:
# 결과를 담을 데이터프레임 생성
cols_result = EQUIP_INFO+CORES+RADIUS+['RANK_INDEX','RANK_INDEX_1','RANK_INDEX_2','PREDICTED_POWER','PREDICTED_POWER2','ERROR']
df_result = pd.DataFrame(columns=cols_result)

In [ ]:
df_result = df_result.astype(
    {
        'IN_RADIUS':'float',
        'OUT_RADIUS':'float',
        'RANK_INDEX':'float',
        'RANK_INDEX_1':'float',
        'RANK_INDEX_2':'float',
        'PREDICTED_POWER':'float',
        'PREDICTED_POWER2':'float',
        'ERROR':'float'
    }
)
df_result.dtypes

### ② 생산조건 입력

최적 공정조건을 찾기 위한 제약조건을 입력합니다.
* 설비(equipment_id)
* 금형 위치(mold_position)
* 목표 도수(target_power)
* 금형 조합(core_group)

In [ ]:
df_sample = df_raw.groupby(['CATEGORY']+EQUIP_INFO+['REAL_POWER','MOLD_IN_TOP']).count().reset_index()
df_sample = df_sample[df_sample['POWER1'] > 10].sort_values(by=['POWER1'], ascending=False)
df_sample = df_sample[df_sample['CATEGORY']=='MYOPIA']
df_sample.reset_index(drop=True, inplace=True)
df_sample.iloc[0:10]

In [ ]:
idx_sample = input("샘플데이터 중 원하는 데이터의 인덱스를 입력해주세요: ")
idx_sample = int(idx_sample)

equipment_id = df_sample.iloc[idx_sample]['EQUIP_ID']
mold_position = df_sample.iloc[idx_sample]['MOLD_POS']
mold_position = int(mold_position)
target_power = df_sample.iloc[idx_sample]['REAL_POWER']
target_power = float(target_power)
core_group = df_sample.iloc[idx_sample]['MOLD_IN_TOP'][:-9]

# 선택한 조건 확인 출력
print(f'equipment_id: {equipment_id}, \
        mold_position: {mold_position},\
        target_power: {target_power}, \
        core_group: {core_group}')

if target_power < 0:
    category = "MYOPIA"
elif target_power == 0:
    category = "BEAUTY"
else:
    category = "HYPER"

### ③ 연관규칙 알고리즘을 통한 금형조합 묶음

####  연관규칙 적용을 위한 데이터 전처리


In [ ]:
# 금형 이름에서 공백(' ')을 밑줄('_')로 변환
df_pre_assoc = df_raw.copy();

df_pre_assoc = df_pre_assoc[df_pre_assoc['MOLD_IN_TOP'].str.contains(core_group)]

for name in CORES:
    df_pre_assoc[name] = df_pre_assoc[name].str.replace(' ', '_')

print(df_pre_assoc[CORES][:5])

#### One hot encoding 실행

In [ ]:
# one hot encoding
df_pre_assoc = pd.get_dummies(df_pre_assoc, columns=CORES)
print(df_pre_assoc.columns[16:], df_pre_assoc.shape)

#### 연관규칙 알고리즘을 위한 데이터셋 추출

In [ ]:
# 전처리 데이터에서 CATEGORY, EQUIP_ID, MOLD_POS, REAL_POWER가 입력조건과 일치하는 데이터만 추출
df_assoc = df_pre_assoc.copy()
assoc_pos = mold_position

df_assoc = df_assoc[(df_assoc['CATEGORY']==category) & 
                    (df_assoc['EQUIP_ID']==equipment_id) & 
                    (df_assoc['MOLD_POS']==mold_position) & 
                    (df_assoc['REAL_POWER']==target_power)]

In [ ]:
# 생산이력이 없으면 입력조건과 CATEGORY가 일치하는 모든 데이터를 대상으로 연관규칙을 실행
if df_assoc.shape[0] == 0:
  print(f'No history, Searching within All')
  df_assoc = df_pre_assoc.copy()
  df_assoc = df_assoc[(df_assoc['CATEGORY']==category) & (df_assoc['REAL_POWER']==target_power)]
  assoc_pos = 0

In [ ]:
df_assoc.shape

In [ ]:
# One-hot 인코딩 결과 int(0, 1)에서 bool(False, True)로 변환
df_ = df_assoc.copy()
df_ex = df_.iloc[:,16:].replace({1:True, 0:False})

print(df_ex.info(verbose=True))

#### 연관규칙 알고리즘 실행

In [ ]:
# 연관규칙 알고리즘
assoc_result = fpgrowth(df_ex, min_support=0.005, use_colnames=True)

# 연관규칙 도출 결과 중 [‘sets’]==4인 경우,
# 즉 4개의 금형이 조합을 이루는 유효한 값을 추려냄 
assoc_result['sets'] = assoc_result['itemsets'].apply(lambda x: len(x))
print(assoc_result.sort_values(by=['sets'], ascending=False))

assoc_result = assoc_result[assoc_result['sets']==4]
print(assoc_result)

### ④ 연관규칙 결과 테이블 만들기

#### 데이터셋의 컬럼명 복원

In [ ]:
# One-hot 인코딩에서 각 금형값 앞에 컬럼명을 추가했던 것을 제거
temp_df = pd.DataFrame(columns=CORES+['SUPPORT'])
temp_list = [list(x) for x in assoc_result['itemsets']]

for i, dset in enumerate(temp_list):
  for name in dset:
    if 'MOLD_IN_TOP_' in name:
      p1 = compile('MOLD_IN_TOP_{}')
      mold_in_top = p1.parse(name)[0]
    elif 'MOLD_IN_BOT_' in name:
      p2 = compile('MOLD_IN_BOT_{}')
      mold_in_bot = p2.parse(name)[0]
    elif 'MOLD_OUT_TOP_' in name:
      p3 = compile('MOLD_OUT_TOP_{}')
      mold_out_top = p3.parse(name)[0]
    elif 'MOLD_OUT_BOT_' in name:
      p4 = compile('MOLD_OUT_BOT_{}')
      mold_out_bot = p4.parse(name)[0]    
  score = assoc_result['support'].iloc[i]

  temp_df.loc[i] = [mold_in_top, mold_in_bot, mold_out_top, mold_out_bot, score]
  temp_df = temp_df.sort_values(by='SUPPORT', ascending=False)

assoc_result = temp_df

# 금형이름에서 '_'를 ' '로 변환
for name in CORES:
    assoc_result[name] = assoc_result[name].str.replace('_', ' ')
print(assoc_result)

#### 비교군 데이터셋 생성

원천 데이터 중 EQUIP_ID와 MOLD_POS가 일치하는 비교군 데이터를 추출한다.

In [ ]:
df_temp_raw = df_raw.copy()
if assoc_pos != 0:
  df_temp_raw = df_temp_raw[(df_temp_raw['EQUIP_ID']==equipment_id) & (df_temp_raw['MOLD_POS']==assoc_pos)]

#### 중복 결과데이터 삭제 (1)
비교군 데이터에서 연관규칙 결과와 일치하는 데이터를 모두 찾고 중복되는 값을 제거한다.

In [ ]:
df_temp_result = pd.DataFrame(columns=df_temp_raw.columns)
df_temp_result = df_temp_result.astype(
    {
        'REAL_POWER':'float',
        'CP':'float',
        'AX':'float',
        'IN_RADIUS':'float',
        'OUT_RADIUS':'float',
        'POWER':'float',
        'POWER1':'float',
        'POWER2':'float',
        'POWER3':'float',
        'POWER4':'float',
        'POWER5':'float'
    }
)

In [ ]:
for i in range(assoc_result.shape[0]):
    # 연관규칙 결과와 일치하는 결과를 모두 찾기
    mold_in_top = assoc_result.loc[i,]['MOLD_IN_TOP']
    mold_in_bot = assoc_result.loc[i,]['MOLD_IN_BOT']
    mold_out_top = assoc_result.loc[i,]['MOLD_OUT_TOP']
    mold_out_bot = assoc_result.loc[i,]['MOLD_OUT_BOT']
    
    df_temp = df_temp_raw[(df_temp_raw['MOLD_IN_TOP']==mold_in_top) & \
                          (df_temp_raw['MOLD_IN_BOT']==mold_in_bot) & \
                          (df_temp_raw['MOLD_OUT_TOP']==mold_out_top) & \
                          (df_temp_raw['MOLD_OUT_BOT']==mold_out_bot)
                         ]
    df_temp_result = pd.concat([df_temp_result, df_temp])
    
# 중복 데이터 제거
df_temp_result = df_temp_result.drop(['MFG_DT','SALE_CD','REAL_POWER','CP','AX','POWER','CATEGORY'], axis=1)

df_temp_result = df_temp_result[EQUIP_INFO+CORES+RADIUS+POWERS]
df_temp_result = df_temp_result.drop_duplicates(CORES+RADIUS)
df_temp_result.reset_index(inplace=True, drop=True)

In [ ]:
df_temp_result[EQUIP_INFO+CORES+POWERS]

#### P 컬럼 생성

POWER1, POWER2, POWER3, POWER4, POWER5열을 pivot 하고 POWER1, POWER2, POWER3, POWER4, POWER5 값을 통합한 컬럼의 이름을 P로 지정

In [ ]:
tmp = df_temp_result.melt(
    id_vars=EQUIP_INFO+CORES,
    value_vars=POWERS,
    value_name='P'
).astype({'P':'float'})

In [ ]:
tmp

#### P값이 중복되는 데이터 삭제

EQUIP_ID, MOLD_POS, core_group이 같은 데이터를 묶고 나머지 값(P 외)의 중위수 찾기

In [ ]:
tmp = tmp.groupby(
    EQUIP_INFO+CORES,
    as_index=False,
    sort=False
).median()

#### 중복 결과데이터 삭제 (2)

EQUIP_ID, MOLD_POS, CORES가 일치하는 데이터끼리 묶고 나머지 값의 중위수를 찾는다.

In [ ]:
df_temp_result = df_temp_result.groupby(
    EQUIP_INFO+CORES,
    as_index=False,
    sort=False
).median()

df_result = pd.concat([df_result, df_temp_result])

#### RANK_INDEX_1 컬럼값 추가
df_result의 'RANK_INDEX_1'열에 P와 target_power의 오차의 절대값을 추가

In [ ]:
df_result['RANK_INDEX_1'] = np.abs(tmp['P'].values - target_power)

In [ ]:
df_result

#### 연관규칙 결과가 존재하지 않을 때
생산이력이 전혀 없는 케이스에 대해서는 프로세스를 종료한다.

In [ ]:
if df_result.shape[0]==0:
    print('입력조건과 일치하는 생산이력이 존재하지 않습니다.')

## [단계 7] 연관규칙 결과와 XGBoost 예측 결과에서 최적조건 찾기

### ① 예측도수 변환에 필요한 변수 및 함수 정의

XGBoost 예측 결과의 POWER1, POWER2, POWER3, POWER4, POWER5로부터 POWER를 할당한다.

In [ ]:
# POWER 할당 함수
POWER_VALUES = np.array([-0.25, -0.5, -0.75,
                         -1.0, -1.25, -1.5, -1.75,
                         -2.0, -2.25, -2.5, -2.75,
                         -3.0, -3.25, -3.5, -3.75,
                         -4.0, -4.25, -4.5, -4.75,
                         -5.0, -5.25, -5.5, -5.75,
                         -6.0, -6.5,
                         -7.0, -7.5,
                         -8.0, -8.5,
                         -9.0, -9.5,
                         -10.0])

def power_allocate(value):
    return POWER_VALUES[np.argmin(abs(POWER_VALUES - value))]

def vectorize(value):
    return np.vectorize(power_allocate)(value)

### ② XGBoost Regressor를 이용한 도수 예측

연관규칙 결과 데이터 df_result에서 IN_RADIUS와 OUT_RADIUS의 값을 부호를 없앤 뒤, IN_RADIUS, OUT_RADIUS, EQUIP_ID, MOLD_POS를 XGBoost 회귀모델에 넣고 REAL_POWER를 예측한다.

In [ ]:
# 생산조건(설비, 금형위치)에 따른 REAL_POWER 예측
df_result[RADIUS] = abs(df_result[RADIUS])

X = df_result[RADIUS+EQUIP_INFO]
X.loc[:,'EQUIP_ID'] = equipment_id
X.loc[:,'MOLD_POS'] = mold_position

X, ENCODER = transform_cols(X,ENCODER)

In [ ]:
# XGBoost Regressor 모델을 이용한 REAL_POWER 예측
df_result['PREDICTED_POWER'] = regressor.predict(X)
df_result['PREDICTED_POWER2'] = df_result['PREDICTED_POWER'].apply(vectorize)

### ③ XGBoost 예측결과 내 우선순위 도출

예측 도수(PREDICTED_POWER)와 목표 도수(target_power)의 오차를 RANK_INDEX_2에 저장한다.

입력조건(equipment_id, mold_postion)과 일치하는 데이터가 있는 경우 최종 우선순위(RANK_INDEX)는 RANK_INDEX_1과 RANK_INDEX_2의 합이 되고, 일치하는 입력조건이 없는 경우는 RANK_INDEX_2를 최종 우선순위로 사용한다.

최종 우선순위가 높은 순으로 최대 5개의 최적조건 도출한다.

In [ ]:
df_result['RANK_INDEX_2'] = np.abs(df_result['PREDICTED_POWER'].values - target_power)
if all(df_result['EQUIP_ID'] == equipment_id) & all(df_result['MOLD_POS'] == mold_position):
    df_result['RANK_INDEX'] = df_result['RANK_INDEX_1'] + df_result['RANK_INDEX_2']
else:
    df_result['RANK_INDEX'] = df_result['RANK_INDEX_2']
df_result = df_result.sort_values(by='RANK_INDEX').iloc[:5] # 5개까지

df_result['EQUIP_ID'] = equipment_id
df_result['MOLD_POS'] = mold_position

### ④ 결과 출력

In [ ]:
df_result['ERROR'] = df_result['PREDICTED_POWER2'] - target_power

In [ ]:
print("\n최종 금형조합 추천 결과")
showing_cols = EQUIP_INFO+CORES+RADIUS+['PREDICTED_POWER','PREDICTED_POWER2','ERROR']
df_result[showing_cols]

In [ ]:
print(f'equipment_id: {equipment_id}, \
        mold_position: {mold_position},\
        target_power: {target_power}, \
        core_group: {core_group}')